In [9]:
import nltk
import wikipedia as wk
import random
import string
import re
import unicodedata #turning accented characters into plain ones
from nltk.corpus import wordnet as wn 
from nltk.stem.wordnet import WordNetLemmatizer #A lemmatizer changes words to their base form.
from collections import defaultdict
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel #These measure how similar two sentences are.

warnings.filterwarnings(action='ignore',category=UserWarning, module="wikipedia")
nltk.download('punkt') #split a paragraph into sentences and split sentences into words.
nltk.download('wordnet') #finding meanings of words, synonyms, and lemmatization 
nltk.download('averaged_perceptron_tagger') #helps the chatbot know what part of speech each word is — noun, verb, adjective, etc.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\z00505yf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\z00505yf\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\z00505yf\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [10]:
import nltk
nltk.data.path.append("C:/Users/z00505yf/nltk_data")
print(nltk.data.find('taggers/averaged_perceptron_tagger/'))

C:\Users\z00505yf\AppData\Roaming\nltk_data\taggers\averaged_perceptron_tagger


In [11]:
data = open('HR.txt','r',errors='ignore')
raw=data.read()
raw=raw.lower()

In [12]:
nltk.download('punkt_tab')
sent_tokens=nltk.sent_tokenize(raw)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\z00505yf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [13]:
print(sent_tokens)

['human resource analytics is at the intersection of three bodies of knowledge:\n\nhuman resource management: sets the meaning and purpose of the analytics.', 'data warehousing: knowing how to process and store hr data efficiently, automation of  collection of data and cleaning data.', 'statistical analysis, presentation and interpretation : helps in translating the identified hr  issues into appropriate analyses and communication of results.', '5 fundamental principles of analytics\n\nhr analytics is about metrics and measurement.', 'good metrics definitions, both narrative and formulaic, and their documentation are key.', 'a professional and good hr analytics person will have the above bodies of knowledge and know their process and intersection.', 'good communication and collaborative skills are essential.', 'the in-depth expertise in your organization is likely to exist in hrm.', 'it.', 'and decision support.', 'you will need to collaborate with these groups.', 'the extent of hr ana

In [14]:
def Normalize(text):
    #remmove punctuation marks
    remove_pun_dict=dict((ord(punct),None) for punct in string.punctuation)
    #word tokenization
    word_token=nltk.word_tokenize(text.lower().translate(remove_pun_dict))
    
    #remove ascii
    new_words=[]
    for word in word_token:
        new_word=unicodedata.normalize('NFKD',word).encode('ascii','ignore').decode('utf-8','ignore') #ascii -  convert to bytes, remove non-ascii #decode - back to string
        new_words.append(new_word)

    #remove tags
    rem=[]
    for w in new_words:
        text=re.sub("&lt;/?.*?&gt;","&lt;&gt;",w)
        rem.append(text)

    #pos tagging and lemmatization
    tag_map = defaultdict(lambda: wn.NOUN)
    tag_map['J']=wn.ADJ
    tag_map['V']=wn.VERB
    tag_map['R']=wn.ADV
    lmtzr=WordNetLemmatizer()
    lemma_list=[]
    rem = [i for i in rem if i]
    for token,tag in nltk.pos_tag(rem):
        lemma=lmtzr.lemmatize(token,tag_map[tag[0]])
        lemma_list.append(lemma)
    return lemma_list
    

In [15]:
welcome_input=('hello', 'hey', 'whats', 'what', 'up')
welcome_response=['hi','hey','I am Glad!']

def welcome(user_response):
    for word in user_response.split():
        if word.lower() in welcome_input:
            return random.choice(welcome_response)

In [16]:
import re
import wikipedia as wk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Define a simple tokenizer if needed
def normalize(text):
    return text.lower().split()  # Simple normalization

def generateResponse(user_response):
    robo_response = ''
    sent_tokens.append(user_response)

    TfidfVec = TfidfVectorizer(stop_words='english')  
    tfidf = TfidfVec.fit_transform(sent_tokens)

    vals = linear_kernel(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]  # takes the most similar value for the user input
    flat = vals.flatten() #converts to 1D array
    flat.sort()
    req_tfidf = flat[-2]

    if req_tfidf == 0 or "tell me about" in user_response.lower():
        print("Checking Wikipedia")
        return wikipedia_data(user_response)
    else:
        robo_response = sent_tokens[idx]  # Fix: Avoid extra space
        return robo_response

def wikipedia_data(input_text):
    reg_ex = re.search(r'tell me about (.*)', input_text, re.IGNORECASE)
    if reg_ex:
        topic = reg_ex.group(1).strip()
        try:
            wiki_summary = wk.summary(topic, sentences=3)
            return wiki_summary
        except Exception:
            return "Sorry, no content found on Wikipedia."
    return "Invalid request."


In [18]:
flag=True
print("My name is Chatter and I'm a chatbot")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response not in ['bye','shutdown','exit', 'quit']):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Chatterbot : You are welcome..")
        else:
            if(welcome(user_response)!=None):
                print("Chatterbot : "+welcome(user_response))
            else:
                print("Chatterbot : ",end="")
                print(generateResponse(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("Chatterbot : Bye!!! ")

My name is Chatter and I'm a chatbot
Chatterbot : I am Glad!
Chatterbot : Checking Wikipedia
Invalid request.
Chatterbot : I am Glad!
Chatterbot : Checking Wikipedia
Invalid request.
Chatterbot : hey
Chatterbot : You are welcome..
